BOTNOI Basic NLP Workshop ครั้งที่ 1

# Get Problem : 

นำ Dataset จาก Truevoice มาทำนายการจำแนกประเภท (Classification) ของ Text 
Credit : https://github.com/PyThaiNLP/truevoice-intent

In [1]:
cd /Users/ruxpolth/Desktop/Botnoi DS Project/truevoice-intent

/Users/ruxpolth/Desktop/Botnoi DS Project/truevoice-intent


In [2]:
ls

Note.txt*       mari_test.csv*  mari_train.csv*


# Get Data :

In [3]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('mari_train.csv')
test_df = pd.read_csv('mari_test.csv')

In [4]:
train_df.head()

,texts,texts_deepcut,action,object,destination
0,ผมไปจ่ายเงินที่ Counter Services เค้าเช็ต . บ...,ผม ไป จ่าย เงิน ที่ Counter Services เค้า เช็...,enquire,payment,billing and payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,internet ยัง ความ เร็ว อยุ่ เท่า ไหร ครับ,enquire,package,promotions
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,ตะกี้ ไป ชำระ ค่า บริการ ไป แล้ว แต่ ยัง ใช้ ...,report,suspend,billing and payment
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,พี่ ค่ะ ยัง ใช้ internet ไม่ ได้ เลย ค่ะ เป็น...,enquire,internet,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,ฮาโหล คะ พอดี ว่า เมื่อ วาน เปิด ซิม ทรูมูฟ แ...,report,phone_issues,billing and payment


In [5]:
train_df['destination'].value_counts() / len(train_df)

billing and payment      0.386970
promotions               0.242600
other queries            0.172888
internet                 0.143906
international dialing    0.025968
true money               0.015534
lost and stolen          0.012134
Name: destination, dtype: float64

In [6]:
train_df['destination'].value_counts() / len(test_df)

billing and payment      1.547281
promotions               0.970025
other queries            0.691286
internet                 0.575402
international dialing    0.103832
true money               0.062114
lost and stolen          0.048517
Name: destination, dtype: float64

จาก Dataset จะเห็นได้ว่ามีการตัดคำด้วย Module ของ Deepcut มาให้แล้ว แต่ในกลุ่มย่อยมีความเห็นว่า ขอลอง Module ตัดคำของ PythaiNLP ที่เป็น Engine newmm จะได้ผลลัพธ์เป็นอย่างไร

In [7]:
#เรียก Module ของ PythaiNLP 
from pythainlp import word_tokenize

In [8]:
# สร้าง Function ในการตัดคำ และ ทำความสะอาดข้อมูล
def get_th_token(text):
    #For Lower/Upper
    text = text.lower()
    #For Remove line breaks
    text = text.replace('\n',' ')
    token = word_tokenize(text)
    return token

In [9]:
# ลองเรียกใช้ Function
get_th_token('ฉันไปโรงเรียน')

['ฉัน', 'ไป', 'โรงเรียน']

In [10]:
# เรียกใช้ Module CountVectorizer เพื่อแปลงข้อความให้เป็นตัวเลข
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
# เแทนที่ Mudule ตัดคำภาษาไทยด้วย Func ที่สร้างไว้ ลงใน Module CountVectorizer และเรียกใช้ N-Gram เพื่อช่วยในการแยกคำ
vectorizer = CountVectorizer(tokenizer=get_th_token, ngram_range=(1,2))

In [12]:
data = ['ฉันไปโรงเรียน','โรงเรียนของเราหน้าอยู่','คุณครูใจดีทุกคน','ปลากินพี่วิน']

In [13]:
vectorizer.fit(data)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function get_th_token at 0x11fc3a5f0>,
                vocabulary=None)

In [14]:
vectorizer.get_feature_names()

['กิน',
 'กิน พี่',
 'ของ',
 'ของ เรา',
 'คุณครู',
 'คุณครู ใจดี',
 'ฉัน',
 'ฉัน ไป',
 'ทุกคน',
 'ปลา',
 'ปลา กิน',
 'พี่',
 'พี่ วิน',
 'วิน',
 'หน้า',
 'หน้า อยู่',
 'อยู่',
 'เรา',
 'เรา หน้า',
 'โรงเรียน',
 'โรงเรียน ของ',
 'ใจดี',
 'ใจดี ทุกคน',
 'ไป',
 'ไป โรงเรียน']

In [15]:
vectorizer.transform(data).toarray()

array([[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 1],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0]])

In [16]:
vectorizer.transform(['โรงเรียนไปโรงเรียน']).toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
        0, 1, 1]])

# Extract Feature

In [17]:
train_data = train_df['texts']
test_data = test_df['texts']

train_label = train_df['destination']
test_label = test_df['destination']

In [18]:
vectorizer = CountVectorizer(tokenizer=get_th_token, ngram_range=(1,2))

In [19]:
vectorizer.fit(train_data)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function get_th_token at 0x11fc3a5f0>,
                vocabulary=None)

In [20]:
train_feat = vectorizer.transform(train_data)
test_feat = vectorizer.transform(test_data)

# Train Model

In [21]:
train_feat.shape

(12939, 32832)

In [22]:
train_feat

<12939x32832 sparse matrix of type '<class 'numpy.int64'>'
	with 318137 stored elements in Compressed Sparse Row format>

In [23]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import normalize

In [24]:
train_feat_norm = normalize(train_feat)
test_feat_norm = normalize(test_feat)

In [25]:
model = LinearSVC(random_state=0, class_weight='balanced')

In [26]:
model.fit(train_feat_norm, train_label)

LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
          verbose=0)

# Evaluetion

In [27]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [28]:
predict = model.predict(test_feat_norm)

In [31]:
print(classification_report(test_label, predict, digits=4))

                       precision    recall  f1-score   support

  billing and payment     0.9271    0.8976    0.9121       977
international dialing     0.9216    0.9261    0.9238       203
             internet     0.8448    0.9203    0.8809       615
      lost and stolen     0.9896    0.9745    0.9820       196
        other queries     0.8630    0.8488    0.8558       549
           promotions     0.9263    0.8949    0.9103       590
           true money     0.9115    0.9717    0.9406       106

             accuracy                         0.9020      3236
            macro avg     0.9120    0.9191    0.9151      3236
         weighted avg     0.9033    0.9020    0.9022      3236



In [30]:
print(confusion_matrix(test_label, predict))

[[877   2  45   1  29  14   9]
 [  6 188   4   0   0   5   0]
 [ 15   0 566   0  23  11   0]
 [  1   0   0 191   4   0   0]
 [ 32   1  36   1 466  12   1]
 [ 15  13  19   0  15 528   0]
 [  0   0   0   0   3   0 103]]


# Conclusion :

จาก Dataset ของ Truevoice หากเราใช้ Module ตัดคำของ PythaiNLP(engine=newmn) --ใช้เทคนิคแปลงคำเป็นตัวเลขด้วย Bag of word และ N-Gram ในที่นี้ใช้เป็น 2-Gram
-- ใช้ L2 Normalize เพื่อแก้ปัญหาความเหมือนกันของ Matrix --ใช้ Model Linear SVC จะได้ accuracy = 90.20%